<a href="https://colab.research.google.com/github/Ruochenge/learning_test/blob/main/SAGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

This Python script modifies a SAGE  configuration JSON file to tailor it to specific parameters for proteomic analysis. The program reads an existing configuration file, updates it with user-specified parameters, and writes the modified configuration to a new JSON file. This process ensures that the analysis settings are customised for particular experimental conditions and requirements.

## Functionality

The main functionality is encapsulated in the `get_sage_config` function, which performs the following steps:

1. **Load Existing Configuration**: Reads the original SAGE configuration JSON file.
2. **Update Configuration**: Modifies the configuration with user-specified parameters such as:
    - Paths to mzML files
    - Variable and static modifications
    - Missed cleavages
    - Enzyme options (restrictions, C-terminal cleavage, semi-enzymatic digestion)
    - Fragment tolerance settings
3. **Save Modified Configuration**: Writes the updated configuration to a new JSON file.

## Parameters

The script allows the user to specify a variety of parameters to customise the proteomic analysis:

- **mzML File Paths**: Paths to the mzML files containing mass spectrometry data.
- **Variable Modifications**: Modifications that can occur on amino acids during the analysis.
- **Static Modifications**: Fixed modifications on amino acids.
- **Missed Cleavages**: The maximum number of cleavages missed by the enzyme.
- **Enzyme Restrictions**: Options to restrict cleavage after specific amino acids, enable C-terminal cleavage, and allow semi-enzymatic digestion.
- **Fragment Tolerance**: Settings for fragment mass tolerance in the analysis.


##[Online documentation](https://sage-docs.vercel.app/docs)


**Precursor Tolerance**:
This parameters specifices whether an absolute ("da") or relative ("ppm") tolerance is used for selecting candidate matches. Sage applies the "left" side of the tolerance to the experimental mass - this means that typical open-searches (i.e looking for a positive delta mass arising from a chemical modification) will want to use a large negative value here. Using the above configuration, an MS2 spectrum with a 2700 Da precursor mass would search for candidate peptides with a mass between 2200 and 2800 Da.

"Closed search" : "ppm": [-20, 20]

"Open search" : "da": [-500, 300]

**Modifications**:
Modification Syntax:
*   "^X": Modification to be applied to amino acid X if it appears at the N-terminus of a peptide
*   "$X": Modification to be applied to amino acid X if it appears at the C-terminus of a peptide
*   "[X": Modification to be applied to amino acid X if it appears at the N-terminus of a protein
*   "]X": Modification to be applied to amino acid X if it appears at the C-terminus of a protein

e.g.

*   "^": 304.207 = N-terminal modification of 304.207
*    "$": 49.2022 = C-terminal modification of 49.2022

###Original paper


*   [Sage: An Open-Source Tool for Fast Proteomics Searching and Quantification at Scale](https://pubs.acs.org/doi/full/10.1021/acs.jproteome.3c00486)  (paper)
* [Proteomics searching so fast it seems like Magic](https://lazear.github.io/sage/) (additional documentation)
*   [YouTube link](https://www.youtube.com/watch?v=1rG-ZE3Z0Xc)



#SAGE

In [ ]:
#@markdown Mount google drive
from google.colab import drive
drive.mount('/content/drive')
from sys import version_info
python_version = f"{version_info.major}.{version_info.minor}"

Mounted at /content/drive


# Conda
Install conda and restart runtime automatically for Sage (wait for 1 min)bold text

In [ ]:
#install conda, kernel will be restarted automatically
!pip install -q condacolab
import condacolab
import time

condacolab.install()
time.sleep(10)

✨🍰✨ Everything looks OK!


##Install sage

In [ ]:
# Check if conda is working
condacolab.check()#check conda is working


✨🍰✨ Everything looks OK!


In [ ]:
!conda install -c bioconda -c conda-forge sage-proteomics

Channels:
 - bioconda
 - conda-forge
Platform: linux-64
Solving environment: | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - sage-proteomics


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    openssl-3.3.0              |       h4ab18f5_3         2.8 MB  conda-forge
    sage-proteomics-0.14.7     |       h031d066_0         7.8 MB  bioconda
    ------------------------------------------------------------
                                           Total:        10.8 MB

The following NEW package

In [ ]:
#!sage --help

# Sage configuration (updating peak lists)




##Functions

In [ ]:
import json
import glob
import os
import re
from datetime import datetime
import subprocess

def ensure_dir_exists(path):
    """Ensure that the directory for the given path exists."""
    directory = os.path.dirname(path)
    if os.path.exists(directory):
        if not os.path.isdir(directory):
            raise NotADirectoryError(f"The path {directory} is not a directory.")
    else:
        os.makedirs(directory)

def get_sage_config(json_file_path, peak_folder, new_mods, missed_cleavages, static_mod, output_config_path, restrict=None, c_terminal=True, semi_enzymatic=None, fragment_tol=None):
    """
    Reads a SAGE configuration JSON file, updates specified options, and writes the modified
    configuration to a new file. Also prints summaries of the configuration for easier inspection.

    Args:
        json_file_path (str): Path to the original SAGE configuration JSON file.
        peak_folder (str): Path to the folder containing mzML files (glob pattern supported).
        new_mods (dict): Dictionary of variable modifications for the analysis.
            Keys are amino acids, values are lists of mass shifts.
        missed_cleavages (int): Maximum number of missed cleavages allowed.
        static_mod (dict): Dictionary of static modifications for the analysis.
            Keys are amino acids, values are mass shifts.
        output_config_path (str): Path to save the modified SAGE configuration JSON file.
        restrict (str, optional): Amino acid to restrict cleavage after. Defaults to None (no restriction).
        c_terminal (bool, optional): Allow C-terminal cleavage. Defaults to True.
        semi_enzymatic (bool, optional): Enable semi-enzymatic digestion. Defaults to None (full enzymatic).
        fragment_tol (dict, optional): Dictionary specifying fragment tolerance settings. Defaults to None.

    Returns:
        dict: The modified JSON data.
    """
    # Ensure the output directory exists
    ensure_dir_exists(output_config_path)

    # Load the existing configuration
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)

    # # Update configuration with provided arguments
    # json_data['mzml_paths'] = glob.glob(peak_folder)
    # json_data['database']['variable_mods'] = new_mods
    # json_data['database']['enzyme']['missed_cleavages'] = missed_cleavages
    # json_data['database']['static_mods'] = static_mod

    # # Update enzyme options if provided
    # if restrict is not None:
    #     json_data['database']['enzyme']['restrict'] = restrict
    # if c_terminal is not None:
    #     json_data['database']['enzyme']['c_terminal'] = c_terminal
    # if semi_enzymatic is not None:
    #     json_data['database']['enzyme']['semi_enzymatic'] = semi_enzymatic

    # # Update fragment tolerance if provided
    # if fragment_tol is not None:
    #     json_data['fragment_tol'] = fragment_tol

    # # Save the modified configuration to a new file
    # with open(output_config_path, 'w') as f:
    #     json.dump(json_data, f, indent=4)

    # return json_data

def save_log_info(SAGE_log_info, file_path):
    """Saves the log information to a text file and prints it."""
    # Ensure the output directory exists
    ensure_dir_exists(file_path)

    with open(file_path, 'w') as SAGE_log_file:
        for line in SAGE_log_info:
            # Remove ANSI escape sequences
            cleaned_line = re.sub(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])', '', line)
            SAGE_log_file.write(cleaned_line + '\n')

def run_sage_command(json_file_path, FASTA_file_path, output_config_path):
    """Runs the SAGE command and captures the output."""
    # Construct the SAGE command dynamically
    sage_command = [
        "sage",
        json_file_path,
        "--fasta", FASTA_file_path,
        "--write-pin", output_config_path
    ]

    # Run the SAGE command and capture the output
    result = subprocess.run(sage_command, capture_output=True, text=True)

    # Combine stdout and stderr
    captured_output = result.stdout + result.stderr

    # Convert captured output to a list of lines
    captured_output_lines = captured_output.splitlines()

    # Print the command output for debugging purposes
    print("\nSAGE Command Output:\n")
    print(result.stdout)
    print(result.stderr)

    return captured_output_lines

# def filter_log_info(captured_output_lines):
#     """Filters out the JSON part from the captured output."""
#     return [line for line in captured_output_lines if not (line.startswith('{') or line.endswith('}') or line.strip().startswith('"'))]


##Inputs

In [ ]:
#@title File details

#@markdown **SAGE input single file**
mzML_file = "\"/content/drive/Shareddrives/MS_data/mzML/Parchment/Calf_mzml/20171204_QE3_nLC7_MEM_COLLAB_SD_UofY_JSK08_2.mzML" #@param {type:"string"}
#@markdown **SAGE directory**
mzML_folder = "/content/drive/Shareddrives/MS_data/mzML/Parchment/Calf_mzml/*.mzML" #@param {type:"string"}

output_config_path = "/content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/CALF/config_modified.json" #@param {type:"string"}
#@markdown - a `.mztab` output from SAGE
json_file_path = '/content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/config_COW_basic_sage.json' #@param {type:"string"}
#@markdown - the `.json` configuration file for SAGE
FASTA_file_path = '/content/drive/MyDrive/Colab_Notebooks/Dictionaries/FASTA_dicts/FASTA/Species/Cow.fasta' #@param {type:"string"}

peak_folder = "/content/drive/Shareddrives/MS_Palaeome/Odense_RAWs/23_Apr_2024/mzML/BS/*.mzML"

In [ ]:
#@title ###Edit json file

# #Fragment Index Construction
semi_enzymatic = False
restrict = None  # No restriction
c_terminal = True  #  Boolean. Cleave at the C-terminus of matching amino acids (default:true).

# Define fragment tolerance for open search  ~ -500,100 suggested in the manual
fragment_tol = {
    "da": [
        -100,
        300
        ]
}
# Define variable modifications (replace with your modifications)
variable_mods = {
    "K": [
        15.9949,
        27.9949,
        162.1406
        ],
    "C": [
        57.0215
        ],
    "^": [
        27.994915,
        43.005814,
        87.03203,
        100.016045
        ],
    "N": [
        0.98402
        ],
    "^E": [
        -17.026548
        ],
    "M": [
        15.9949
        ],
    "P": [
        13.9793,
        15.9949,
        16.9902
        ],
    "Q": [
        0.984016
        ],
    "A": [
        58.0055
        ],
    "^Q": [
        -17.02655
        ]
}

# static_mods = {
#     "C":
#     0
# }

missed_cleavages = 2

#Run Sage

In [ ]:
# # Call the function with example arguments (paths should be adjusted to your environment)
# json_data = get_sage_config(json_file_path, mzML_folder, variable_mods, missed_cleavages, static_mods, output_config_path, restrict=restrict, c_terminal=c_terminal, semi_enzymatic=semi_enzymatic, fragment_tol=fragment_tol)

# # Verify the saved JSON data
# with open(output_config_path, 'r') as f:
#     saved_json_data = json.load(f)
#     print("Saved JSON data:")
#     print(json.dumps(saved_json_data, indent=4))


# Run the SAGE command and capture the output
captured_output_lines = run_sage_command(json_file_path, FASTA_file_path, output_config_path)


# Define log information and the file path for saving logs
SAGE_log_info = captured_output_lines
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
log_file_path = f"/content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/CALF/SAGE_log_info_{current_time}.txt"

# Save log information to the text file with the new filename
save_log_info(SAGE_log_info, log_file_path)

print(f"Log information saved to: {log_file_path}")


SAGE Command Output:


Error: Failed to read parameters from `/content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/config_COW_basic_sage.json`

Caused by:
    expected `,` or `}` at line 70 column 3

Log information saved to: /content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/CALF/SAGE_log_info_20240601_184236.txt


In [ ]:
# Open the saved file for reading
with open(log_file_path, 'r') as SAGE_log_file:
  saved_log_content = SAGE_log_file.read()

# Print the saved log content
print("Saved log information:")
print(saved_log_content)

Saved log information:
[2024-06-01T18:29:58Z WARN  sage::input] Tolerances are applied to experimental masses, not theoretical: [-100, 500]



In [ ]:
!sage /content/drive/MyDrive/Colab_Notebooks/MS_output/SAGE/config_COW_sage.json > output.tsv

[2024-06-01T13:54:28Z INFO  sage] generated 1833501 fragments, 55151 peptides in 222ms
[2024-06-01T13:54:28Z INFO  sage] processing files 0 .. 1 
[2024-06-01T13:54:35Z INFO  sage] - file IO:     7210 ms
[2024-06-01T13:54:40Z INFO  sage] - search:      5612 ms (4468 spectra/s)
[2024-06-01T13:54:41Z INFO  sage_core::ml::retention_alignment] aligning file #0: y = 1.0000x + 0.0000
[2024-06-01T13:54:41Z INFO  sage_core::ml::retention_alignment] aligned retention times across 1 files
[2024-06-01T13:54:41Z INFO  sage_core::ml::retention_model] - fit retention time model, rsq = 0.5496721640157476
[2024-06-01T13:54:41Z INFO  sage_core::ml::mobility_model] - fit mobility model, rsq = NaN, mse = 0
[2024-06-01T13:54:41Z WARN  sage] linear model fitting failed, falling back to heuristic discriminant score
[2024-06-01T13:54:41Z INFO  sage] discovered 2384 target peptide-spectrum matches at 1% FDR
[2024-06-01T13:54:41Z INFO  sage] discovered 0 target peptides at 1% FDR
[2024-06-01T13:54:41Z INFO  sag

#Run Sage in the shell

#Sage clean

# Run Sage with a big fasta (>= 100k entries, @Yun's code)

In [ ]:
# """
# run Sage and a fasta in batches (when a fasta >= 100k entries)
# slices=integer (choose how many slices for an input fasta)
# input_fasta=path to a fasta
# input_json=path to a json configuration file
# output_folder=a folder path to save outputs

# """

# #Change inputs here
# slices=5
# input_fasta="/content/drive/MyDrive/active/orthrus_final/nature/nature_v1.fasta"
# input_json="/content/drive/MyDrive/active/orthrus_final/nature/modern_stool_841376/nature_gut_841376.json"
# output_folder="/content/drive/MyDrive/active/orthrus_final/nature/v1"


# #now click the play button to run the cell
# import pandas as pd
# import subprocess
# from Bio.Seq import Seq
# from Bio.SeqRecord import SeqRecord
# from Bio import SeqIO



# def run_DB_in_batches (slices, input_fasta, input_json, output_folder):

#   records = []

#   for record in SeqIO.parse(input_fasta, format="fasta"):
#     records.append(record)

#   N = len(records) // slices + (len(records) % slices > 0)

#   for i in range(slices):

#     fasta_slice_path = f"{output_folder}/input_fasta_slice_{i}.fasta"

#     with open(fasta_slice_path, "w") as f:
#       for record in records[i * N : (i + 1) * N]:
#         SeqIO.write(record, f, format="fasta")

#     cmd = [
#         "sage",
#         input_json,
#         "-o",
#         f"{output_folder}/output_slice_{i}",
#         "-f",
#         fasta_slice_path,
#         "--write-pin"
#     ]
#     subprocess.run(cmd)

#   dfs = []
#   for i in range(slices):
#     results_path = f"{output_folder}/output_slice_{i}/results.sage.tsv"
#     dfs.append(pd.read_table(results_path))

#   pd.concat(dfs).sort_values(by="hyperscore", ascending=False).drop_duplicates(
#     subset=["filename", "scannr"], keep="first").to_csv(f"{output_folder}/combined.tsv", sep="\t", index=False)



# run_DB_in_batches (slices, input_fasta, input_json, output_folder)

##Clean Sage

In [ ]:
#clean sage output tsvs
import pandas as pd
def sage_clean (df):
  s=df.copy()
  s1=s.assign(decoy=s['proteins'].str.contains('rev', regex=False), seq_naked=s['peptide'].str.replace('\[\+[\d\.]+\]', '', regex=True))
  s2=s1[s1['decoy']==False]
  return s2

df=pd.read_table('/content/drive/MyDrive/active/PK4HJ1_RB1_1_14436/orthrus_results/results.sage.tsv')
df1=sage_clean(df)

In [ ]:
df1['proteins'].value_counts().head(50)

#Mokapot

In [ ]:
!pip install mokapot

In [ ]:
import mokapot
psms = mokapot.read_pin("/content/drive/MyDrive/active/PK4HJ1_RB1_1_14436/orthrus_results/results.sage.pin")
results, models = mokapot.brew(psms)
results.to_txt("/content/drive/MyDrive/active/PK4HJ1_RB1_1_14436/orthrus_results")

In [ ]:
import pandas as pd
temp=pd.read_table('/content/drive/MyDrive/active/PK4HJ1_RB1_1_14436/orthrus_results/mokapot.psms.txt')
temp['Proteins'].value_counts().head(50)

In [ ]:
df1['proteins'].value_counts().head(15)

In [ ]:
df2=df1[df1['spectrum_q']<=0.01]
df2

In [ ]:
df2['proteins'].value_counts().head(40)

In [ ]:
def collect_all (files_path):
  files=os.path.join(files_path, "*.tsv")
  files=glob.glob(files)
  c=pd.concat(map(pd.read_table, files), axis=0, ignore_index=True)
  return c

In [ ]:
df2=collect_all('/content/drive/MyDrive/active/orthrus_final/orthrus_old/O_host/natureGut/ranked/n_tsvs')
df3=sage_clean(df2)

In [ ]:
df4=df3.sort_values(by="hyperscore", ascending=False).drop_duplicates(subset=["filename", "scannr"], keep="first")

In [ ]:
df5=df4[df4['spectrum_q']<=0.01]

In [ ]:
df5

In [ ]:
df5['proteins'].value_counts().head(10)

In [ ]:
t=pd.read_table('/content/drive/MyDrive/active/orthrus_final/orthrus_old/O_host/natureGut/gut_host_evidence.txt')
t.columns